# TheSparkSession

In [2]:
# Show the details of the SparkSession
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.3 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [3]:
# Create a DataFrame and perform an action
myRange = spark.range(1000).toDF("number")
myRange.show(5)

+------+
number|
+------+
 0|
 1|
 2|
 3|
 4|
+------+
only showing top 5 rows

In [4]:
# Create a DataFrame and perform a transformation and an action
divisBy2 = myRange.where("number % 2 = 0")
divisBy2.show(5)

+------+
number|
+------+
 0|
 2|
 4|
 6|
 8|
+------+
only showing top 5 rows

# An End-to-End Example

In [6]:
# Load the data (a transformation)
flightData2015 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv")

# Show some rows (an action)
flightData2015.take(3)

Out[ 17 ]: [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [7]:
# Sort the rows (a transformation) and look at the explain plan
flightData2015.sort("count").explain()

== Physical Plan ==
Sort [count#170 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#170 ASC NULLS FIRST, 200)
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#168,ORIGIN_COUNTRY_NAME#169,count#170] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>

In [8]:
# Create a table
flightData2015.createOrReplaceTempView("flight_data_2015")

In [9]:
# Execute a command in the sql way
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")
sqlWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#168], functions=[finalmerge_count(merge count#185L) AS count(1)#180L])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#168, 200)
 +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#168], functions=[partial_count(1) AS count#185L])
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#168] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>

In [10]:
# Execute a command in the DataFrame way
dataFrameWay = flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .count()
dataFrameWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#168], functions=[finalmerge_count(merge count#197L) AS count(1)#192L])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#168, 200)
 +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#168], functions=[partial_count(1) AS count#197L])
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#168] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>

We get the same explain plan!

In [12]:
# Execute another action in the DataFrame
from pyspark.sql.functions import max
flightData2015.select(max("count")).take(1)

Out[ 25 ]: [Row(max(count)=370002)]

In [13]:
# Execute a more complex command in the sql way
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

maxSql.show()

+-----------------+-----------------+
DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
 United States| 411352|
 Canada| 8399|
 Mexico| 7140|
 United Kingdom| 2025|
 Japan| 1548|
+-----------------+-----------------+

In [14]:
# Execute it in the DataFrame way
from pyspark.sql.functions import desc

flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .show()

+-----------------+-----------------+
DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
 United States| 411352|
 Canada| 8399|
 Mexico| 7140|
 United Kingdom| 2025|
 Japan| 1548|
+-----------------+-----------------+

In [15]:
# And check the new explain plan
flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#292L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#168,destination_total#292L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#168], functions=[finalmerge_sum(merge sum#296L) AS sum(cast(count#170 as bigint))#288L])
 +- Exchange hashpartitioning(DEST_COUNTRY_NAME#168, 200)
 +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#168], functions=[partial_sum(cast(count#170 as bigint)) AS sum#296L])
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#168,count#170] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>